<h1><center> <b>Google Summer of Code 2021 </b></center></h1>

![alt text](images/gsoc.png)

**Organization:** Machine Learning for Science (ML4SCI) Umbrella Organization <br/>
**Project Title:** Background Estimation with Neural Autoregressive Flows <br/>
**Code author:** Sinan Gençoğlu | sinan.gencogluu@gmail.com

### Project Definition

* Data-driven background estimation is crucial for many scientific searches, including searches for new phenomena in experimental datasets. Neural autoregressive flows (NAF) is a deep generative model that can be used for general transformations and is therefore attractive for this application. The MLBENDER project focuses on studying how to develop such transformations that can be learned and applied to a region of interest. In this project the main aim is implementing a Neural Autoregressive Flow (NAF) model to estimate the background distribution and apply it to a representative physics analysis searching for a resonance excess over a smooth background.

##### Terminology: Normalizing Flows 

* In order to estimate the probability of some distributions(in this case: background distributions of events), we can use some generative models that are capable of estimating the probabilities. Normalizing flow models are powerful distributions approximaters. Basically,  a normalizing flow transforms a simple distribution into a complex one by applying a sequence of invertible transformation functions.

![alt-text](images/nf.png)

<center>Reference: <a href="Reference from:">https://lilianweng.github.io/lil-log/2018/10/13/flow-based-deep-generative-models.html </a></center><br>

* In our case I chose to implement Masked Autoregressive Flow [MAF  Papamakarios et al., 2017](https://arxiv.org/abs/1705.07057). In addition to that, I also had another normalizing flow model, RealNVP, in case I couldn't get enough performance from MAF. I do not go into detail about RealNVP because it is out of scope for this part of the study.

* MAF tries to generate samples conditioned on previous ones. This is a sequential process. You can see representation of the generation below. 


<center><b>Conditional Generation Process of the Masked Autoregressive Flow</b></center><br>

![alt-text](images/maf.png)
<center>Reference: <a href="Reference from:">https://blog.evjang.com/2018/01/nf2.html </a></center><br>

* The gray unit $x_i$ is the unit we are trying to compute, and the blue units are the values it depends on. $α_i$ and $μ_i$ are scalars that are computed by passing $x_{1:i−1}$ through neural networks

### Dataset

* I have used a couple of datasets during the studies. For example, first dataset is [Large Hadron Collider (LHC) Dataset](https://zenodo.org/record/2629073). It consists of ~1M jet events and events contains some particles. For each event, all particles in the event are assumed to be massless and are recorded in detector coordinates (pT, eta, phi). These are represented as a DataFrame and each event consist of 21K+1 features (last feature defines that whether the event is coming from the background or signal part of the data, other features is presented in groups of 3 consecutive features. Details can be found on the previous link). Other dataset is relatively same as before but this time we have 76+1 features to estimate.

<center> <b>Sample of the dataset</b></center>

![alt-text](images/data2_1.png)

![alt-text](images/data_1.jpeg)

* All features represent a jet. So, we have 76 jets and 1 feature that defines whether the jets are coming from background or signal.

### Pre-processing steps

* First we import the neccessary packages

In [2]:
# IMPORTS # 

from torch.utils.data import Dataset
import pandas as pd
import torch

import argparse
import copy
import math
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import MAF as fnn
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# We load the data.
df = pd.read_parquet("TTTT_DNN_nJ4_nB2_2018.parquet")

In [4]:
# If you want to use background based events you can change type: 0 for background, 1 for signal
data_type = 0 # background
data = df.loc[df[df.columns.values[-1]]==data_type]
data.shape

(1804642, 77)

In [5]:
data.head()

,AK4HTpMETpLepPt,minMleppBjet,mass_minBBdr,deltaR_lepBJet_maxpt,lepDR_minBBdr,centrality,deltaEta_maxBB,aveCSVpt,aveBBdr,FW_momentum_0,...,HOTGoodTrijet1_dRtridijet,HOTGoodTrijet1_csvJetnotdijet,HOTGoodTrijet1_dRtrijetJetnotdijet,HOTGoodTrijet2_mass,HOTGoodTrijet2_dijetmass,HOTGoodTrijet2_pTratio,HOTGoodTrijet2_dRtridijet,HOTGoodTrijet2_csvJetnotdijet,HOTGoodTrijet2_dRtrijetJetnotdijet,type
231977,682.360291,76.110710,51.740883,1.891567,1.964009,0.460657,-2.927261,0.626476,2.518530,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
231978,913.399109,102.672829,854.941467,4.175213,3.589615,0.515980,3.121105,0.694751,3.202137,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
231979,788.331055,68.620819,82.993179,0.494603,2.828682,0.366740,-1.837404,0.986366,2.461292,1.0,...,0.126494,0.020007,0.918655,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
231980,757.287476,120.900139,220.880264,1.906771,2.003259,0.587130,1.707826,0.847611,2.374820,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
231981,677.532288,39.759781,131.753311,0.678705,3.126003,0.777935,-1.066363,1.004760,3.194130,1.0,...,0.123956,0.045146,0.893217,184.401566,92.719444,0.83411,0.317127,0.998275,1.508981,0.0


In [6]:
# We can take a look at the all features(jets)
for i, col in enumerate(data.columns):
    print(i, col)

0 AK4HTpMETpLepPt
1 minMleppBjet
2 mass_minBBdr
3 deltaR_lepBJet_maxpt
4 lepDR_minBBdr
5 centrality
6 deltaEta_maxBB
7 aveCSVpt
8 aveBBdr
9 FW_momentum_0
10 FW_momentum_1
11 FW_momentum_2
12 FW_momentum_3
13 FW_momentum_4
14 FW_momentum_5
15 FW_momentum_6
16 mass_maxJJJpt
17 BJetLeadPt
18 deltaR_minBB
19 minDR_lepBJet
20 MT_lepMet
21 AK4HT
22 hemiout
23 theJetLeadPt
24 corr_met_MultiLepCalc
25 leptonPt_MultiLepCalc
26 mass_lepJets0
27 mass_lepJets1
28 mass_lepJets2
29 MT2bb
30 mass_lepBJet0
31 mass_lepBJet_mindr
32 secondJetPt
33 fifthJetPt
34 sixthJetPt
35 PtFifthJet
36 mass_minLLdr
37 mass_maxBBmass
38 deltaR_lepJetInMinMljet
39 deltaPhi_lepJetInMinMljet
40 deltaR_lepbJetInMinMlb
41 deltaPhi_lepbJetInMinMlb
42 M_allJet_W
43 HT_bjets
44 ratio_HTdHT4leadjets
45 csvJet3
46 csvJet4
47 firstcsvb_bb
48 secondcsvb_bb
49 thirdcsvb_bb
50 fourthcsvb_bb
51 NJets_JetSubCalc
52 HT_2m
53 Sphericity
54 Aplanarity
55 minDR_lepJet
56 BDTtrijet1
57 BDTtrijet2
58 BDTtrijet3
59 BDTtrijet4
60 NresolvedTo

### Normalizing the data

* Normalization is a technique often applied as part of data preparation for machine learning. The goal of normalization is to change the values of numeric columns in the dataset to a common scale, without distorting differences in the ranges of values. 

* In our case, our dataset have so many features that have different range of values. We need to normalize these values in order to get good results. Since there are too many variables with variable values in the dataset, it is very difficult to generate the background distributions of all the features one by one by applying a single normalization method. Therefore, for a more efficient training, I divided the data into 3 with similar value range features together. First part contains attributes with continuous values, second part contains features with discrete values, and the last part contains the features with sparse values. I will show the one part of the results, you can generate samples for the other parts easily.

* Before I got my final results, I have used couple of normalizations but I will show the one that works reasonably well in this case which is MinMaxScaling. 

In [7]:
# Simple dataloader for training the model.

class TTTT_Data(Dataset):
    def __init__(self, data):
        self.data = data
        self.len = len(self.data)
    
    def __getitem__(self, index):
        x = torch.FloatTensor(self.data[index])
        return x
    
    def __len__(self):
        return self.len

In [9]:
# Normalization 

scaler = MinMaxScaler()
train_data = scaler.fit_transform(data.values)
train_data = TTTT_Data(train_data)

* It is crucial to keep our scaler out of the TTTT_Data class, we will need it in the further processes. Since our dataset has different range of values. For example, if it contains images we can simply divide it by 255 then denormalize it by multiplying with 255. But in this case it is not possible and we have to use inverse of the function that we used in the normalization part.

### Hyperparameters

In [10]:
num_inputs = 8 # Feature size
num_hidden = 64 # Number of hidden layers
num_cond_inputs = False
learning_rate = 1e-3
batch_size = 64
block_size = 7

# Optimizers
gamma = 0.1
step_size = 5 # After 5 epoch decrease learning rate by gamma.

act = 'relu'

### Training Settings

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = TTTT_Data(train_data)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)


modules = []

for _ in range(block_size):
    modules += [
        fnn.MADE(num_inputs, num_hidden, num_cond_inputs, act=act),
        fnn.BatchNormFlow(num_inputs),
        fnn.Reverse(num_inputs)
    ]

model = fnn.FlowSequential(*modules)
model.train()

for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.orthogonal_(module.weight)
        if hasattr(module, 'bias') and module.bias is not None:
            module.bias.data.fill_(0)

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

loss_training = []

print("Model is Cuda?", next(model.parameters()).is_cuda)

def train(epoch, loss_training):
    model.train()
    train_loss = 0

    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        loss = -model.log_probs(data, None).mean()
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if (batch_idx+1)%2000==0:
          print("Loss:", train_loss/(batch_idx+1))
          print("Iteration:", batch_idx+1)
    
    loss_training.append(train_loss/len(train_loader.dataset))
    scheduler.step()

for epoch in range(10):
    print('\nEpoch: {}'.format(epoch+1))
    train(epoch, loss_training)
torch.save(model.state_dict(), 'model' + '.pth')

* Training process can take some time. These are the lost curve using only small part of the data.

<center><b>Loss vs Epoch Curve</b></center>

![](images/loss.png)

### Generating Samples 

* To generate samples from the model you can switch to evaluation mode by using **model.eval()** then:

In [ ]:
num_samples = 5000

# Generate random samples
fixed_noise = torch.Tensor(num_samples, num_inputs).normal_()

# Get model predictions
samples = model.sample(num_samples, noise=fixed_noise).detach().cpu().numpy()

# Denormalize samples
normalized = scaler.inverse_transform(samples)

* After generating samples from the model you can compare how your model works.

In [ ]:
fig = plt.figure(figsize=(14, 14))
rows = 4
cols = 2
axes=[]
for i in range(rows*cols):
    axes.append(fig.add_subplot(rows, cols, i+1))
    label = str(data.columns[i])[1:]
    plt.xlabel(label)
    sns.distplot(data.values[:, i])
    sns.distplot(normalized[:, i])

fig.tight_layout() 
fig.legend(labels=['Real_data','Random_generated'])

![alt-text](images/dists.png)

![alt-text](images/results.png)